**CSI 5139 Assignment 3**

This assignment explores transfer learning.

# Setup

In [68]:
import os
from shutil import copy

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.optimizers import RMSprop, Adagrad, Adam, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model, Model

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix

from pprint import pprint

# VGG-16

In [66]:
#datagen = ImageDataGenerator(rescale=1./255)
datagen = ImageDataGenerator()


train_batchsize = 32 # 160 images per 68 classes
test_batchsize = 4 # 20 images per 68 classes

train_generator = datagen.flow_from_directory(
    directory=r"C:/Users/Sophie/Documents/GitHub/CSI5139/csi5139_a2-Copy/Outex-TC-00030/train/",
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=train_batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = datagen.flow_from_directory(
    directory=r"C:/Users/Sophie/Documents/GitHub/CSI5139/csi5139_a2-Copy/Outex-TC-00030/test/",
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=test_batchsize,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

print("Train generator length: ", len(train_generator))
print("Test generator length: ", len(test_generator))
print("Target size: ", train_generator.target_size)
#print("Generator indices: ", train_generator.class_indices)
#print("Generator filenames:", train_generator.filenames[:10])

Found 10880 images belonging to 68 classes.
Found 1360 images belonging to 68 classes.
Train generator length:  340
Test generator length:  340
Target size:  (128, 128)


#### Model

In [69]:
epochs = 1

vgg_conv = vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
vgg_conv.summary()

#vgg_conv.layers = vgg_conv.layers[-9]
#vgg_conv = Model(vgg_conv.input, vgg_conv.layers[-5].output)

print(vgg_conv.summary())

for layer in vgg_conv.layers:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

model = Sequential()
model.add(vgg_conv)
#model.add(Conv2D(filters = 512, kernel_size = 5, strides = 1, activation = 'relu', input_shape = (16,16,256)))
#model.add(MaxPooling2D(pool_size = 2, strides = 2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(68, activation = 'softmax'))

model.layers[0].trainable = False

for layer in model.layers:
    print(layer, layer.trainable)

model.summary()

rmsprop = RMSprop()
#sgd = SGD(lr=0.008)
#sgd = SGD()

checkpoint_dir = os.path.dirname("./runs/vgg16_2.ckpt")

cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [70]:
hist_vgg16 = model.fit_generator(
        train_generator,
        steps_per_epoch=(train_generator.samples/train_generator.batch_size),
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=(test_generator.samples/test_generator.batch_size),
        verbose=1,
        callbacks = [cp_callback])

Epoch 1/1
339/340 [============================>.] - ETA: 4s - loss: 3.3613 - acc: 0.6593
Epoch 00001: saving model to ./runs/vgg16.ckpt
340/340 [==============================] - 1894s 6s/step - loss: 3.3555 - acc: 0.6597 - val_loss: 2.4253 - val_acc: 0.7684


In [71]:
# save history
hist_vgg16 = hist_vgg16.history
with open('./runs/vgg16.hist', 'w', encoding="utf-8") as fout:
    pprint(hist_vgg16, fout)